In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import feature_column as fc
from tensorflow.keras.datasets import boston_housing

In [2]:
tf.keras.backend.set_floatx('float64')

In [3]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [4]:
features = ['CRIM', 'ZN', 
            'INDUS','CHAS','NOX','RM','AGE',
            'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

x_train_df = pd.DataFrame(x_train, columns= features)
x_test_df = pd.DataFrame(x_test, columns= features)
y_train_df = pd.DataFrame(y_train, columns=['MEDV'])
y_test_df = pd.DataFrame(y_test, columns=['MEDV'])

x_train_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65


In [5]:
feature_columns = []
for feature_name in features:
    feature_columns.append(fc.numeric_column(feature_name,dtype=tf.float32))

In [6]:
def estimator_input_fn(df_data,df_label,epochs=10,shuffle=True,batch_size=32):
    def input_fn():
        ds = tf.data.Dataset.from_tensor_slices((dict(df_data),df_label))
        if shuffle:
            ds = ds.shuffle(100)
        ds = ds.batch(batch_size).repeat(epochs)
        return ds
    return input_fn

train_input_fn = estimator_input_fn(x_train_df,y_train_df)
val_input_fn = estimator_input_fn(x_test_df,y_test_df,epochs= 1, shuffle= False)

In [10]:
linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns,model_dir='./logs/func')
linear_est.train(train_input_fn,steps=100)
result = linear_est.evaluate(val_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './logs/func', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:

In [11]:
result = linear_est.predict(val_input_fn)
for pred,exp in zip(result, y_test[:32]):
    print("predicted value ", pred['predictions'][0], "expected : ", exp )

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./logs/func/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
predicted value  4.6925993 expected :  7.2
predicted value  25.431232 expected :  18.8
predicted value  23.49579 expected :  19.0
predicted value  24.446207 expected :  27.0
predicted value  24.197111 expected :  22.2
predicted value  23.488005 expected :  24.5
predicted value  31.432545 expected :  31.2
predicted value  27.277107 expected :  22.9
predicted value  22.365612 expected :  20.5
predicted value  26.036306 expected :  23.2
predicted value  10.209732 expected :  18.6
predicted value  24.755768 expected :  14.5
predicted value  25.24839 expected :  17.8
predicted value  24.514202 expected :  50.0
predicted value  20.373173 expected :  20.8
predicted value  26.456345 expected :  24.3
predicted value  22.08092 expected : 

In [15]:
import os
logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)
%load_ext tensorboard
%tensorboard --logdir {logs_base_dir}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 14301), started 0:01:43 ago. (Use '!kill 14301' to kill it.)